In [18]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import pandas as pd

from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models
from PIL import Image
from torch.autograd import Variable
from typing import Union
from PIL import Image
from torch import Tensor

In [ ]:
# root_dir_test = '/Users/vionnietan/Desktop/trial_dataset/coad_msi_mss/test'
root_dir_test = 'C:/Users/jones/Desktop/cancer dataset/coad_msi_mss/test'
data_transformation_test = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

test_image_dataset = datasets.ImageFolder(root = root_dir_test, transform=data_transformation_test)
testloader = DataLoader(test_image_dataset, batch_size=128, shuffle=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = torch.load('/Users/vionnietan/Desktop/resnet18.pth')
model = torch.load('C:/Users/jones/Desktop/FIT3164/22septmodel.pth')
model.eval()
model

In [ ]:
class_names = testloader.dataset.classes

def show_images(images, labels, preds):
    plt.figure(figsize=(8,4))
    for i, image in enumerate(images):
        if i < 5:
            plt.subplot(1, 6, i+1, xticks=[], yticks=[])
            image = image.numpy().transpose((1, 2, 0))
            
            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            
            image = image*std + mean
            image = np.clip(image, 0.,1.)
            plt.imshow(image)
            
            colour = 'green' if preds[i] == labels[i] else 'red'
            plt.xlabel(f'{class_names[int(labels[i].numpy())]}')
            plt.ylabel(f'{class_names[int(preds[i].numpy())]}', color=colour)
    plt.tight_layout()        
    plt.show()

In [ ]:
def show_preds():
    model.eval()    # set to evaluation mode
    images, labels = next(iter(testloader))
    outputs = model(images)
    _ , preds = torch.max(outputs, 1)
    #print(labels)
    show_images(images, labels, preds)

In [ ]:
show_preds()

In [19]:
def to_numpy(tensor: Union[Tensor, Image.Image, np.array]) -> np.ndarray:
    if type(tensor) == np.array or type(tensor) == np.ndarray:
        return np.array(tensor)
    elif type(tensor) == Image.Image:
        return np.array(tensor)
    elif type(tensor) == Tensor:
        return tensor.cpu().detach().numpy()
    else:
        raise ValueError()

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

def test_label_predictions(model, device, test_loader):
    model.eval()
    actuals = []
    predictions = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            prediction = outputs.argmax(dim=1, keepdim=True)
            actuals.extend(to_numpy(labels.view_as(prediction)))
            predictions.extend(to_numpy(prediction))
            
    return [i.item() for i in actuals], [i.item() for i in predictions]

actuals, predictions = test_label_predictions(model, device, testloader)
print('Confusion matrix for resnet18: ')
print(confusion_matrix(actuals, predictions))
print('AUC score for model resnet18: '+str(roc_auc_score(actuals,predictions)))

Confusion matrix for resnet18: 
[[224 210]
 [129 523]]
AUC score for model resnet18: 0.659138135760934
